# Week 1 Seminar: Measuring the Lies in Your Data

The lecture told you that financial data lies. This seminar is where you catch it in the act.

You'll run three forensic investigations on real market data: measuring how much survivorship bias inflates backtest returns, dissecting how corporate actions silently corrupt price histories, and stress-testing a 20-stock dataset for quality issues that a naive analysis would never catch. Each exercise produces specific numbers — and some of those numbers will make you rethink every backtest you've ever seen.

All data for the entire seminar is downloaded in the setup cells below. Run them once, then work through each exercise. Try the tasks yourself before peeking at the solutions — the "aha" moment hits harder when you've wrestled with the data first.

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from pathlib import Path

plt.rcParams.update({"figure.figsize": (12, 6), "axes.grid": True, "grid.alpha": 0.2})
CACHE_DIR = Path("code/.cache")
CACHE_DIR.mkdir(exist_ok=True)

We need data for three exercises covering different facets of data quality: S&P 500 constituents for the survivorship investigation, 5 tickers with known corporate actions, and a broad 20-ticker panel for the quality gauntlet. Everything downloads upfront and caches to Parquet so reruns are instant.

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
ua = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"
sp500_table = pd.read_html(url, storage_options={"User-Agent": ua})[0]
sp500_tickers = sp500_table["Symbol"].str.replace(".", "-", regex=False).tolist()

cache_sp = CACHE_DIR / "sp500_survivorship.parquet"
if cache_sp.exists():
    sp500_prices = pd.read_parquet(cache_sp)
else:
    sp500_prices = yf.download(
        sp500_tickers, start="2010-01-01", end="2025-01-01",
        auto_adjust=True, progress=False)["Close"]
    sp500_prices.to_parquet(cache_sp)

print(f"S&P 500: {sp500_prices.shape[1]} tickers, "
      f"{sp500_prices.index[0].date()} to {sp500_prices.index[-1].date()}")

That's the survivorship data. Next, the corporate-action panel: 5 tickers downloaded with `auto_adjust=False` to get both Close and Adj Close, plus their split histories from yfinance's ticker API.

In [ ]:
CA_TICKERS = ["AAPL", "TSLA", "GE", "NVDA", "JNJ"]
cache_ca = CACHE_DIR / "corporate_actions.parquet"
cache_spl = CACHE_DIR / "corporate_actions_splits.parquet"
if cache_ca.exists():
    ca_data = pd.read_parquet(cache_ca)
    all_splits = pd.read_parquet(cache_spl)
else:
    ca_data = yf.download(CA_TICKERS, start="2010-01-01", end="2025-01-01",
                          auto_adjust=False, progress=False)
    ca_data.to_parquet(cache_ca)
    sd = {}
    for t in CA_TICKERS:
        try:
            s = yf.Ticker(t).splits
            if len(s) > 0: s.index = s.index.tz_localize(None)
            sd[t] = s
        except Exception:
            sd[t] = pd.Series(dtype=float)
    all_splits = pd.DataFrame(sd)
    all_splits.to_parquet(cache_spl)

Finally, the 20-ticker panel for the data quality gauntlet — a broad cross-section spanning large-cap tech, ETFs, healthcare, and industrials so that quality issues in less liquid periods surface naturally.

In [ ]:
DQ_TICKERS = ["AAPL", "MSFT", "JPM", "TSLA", "SPY", "QQQ", "IWM", "XLE",
              "GE", "META", "AMZN", "GOOG", "BRK-B", "V",
              "UNH", "PFE", "INTC", "BA", "DIS", "NFLX"]
cache_dq = CACHE_DIR / "data_quality_test.parquet"
if cache_dq.exists():
    dq_data = pd.read_parquet(cache_dq)
else:
    dq_data = yf.download(DQ_TICKERS, start="2010-01-01", end="2025-01-01",
                          auto_adjust=False, progress=False)
    dq_data.to_parquet(cache_dq)

## Exercise 1: The Survivorship Trap — Measuring Ghost Returns

Here's an experiment you can run in five minutes that will permanently change how you read backtest results. Go to any quantitative finance blog, find a backtest that uses "S&P 500 stocks" as its universe, and ask: *which* S&P 500 stocks? Today's list? The list from the backtest start date? Those are very different lists. Companies that went bankrupt, were acquired, or were delisted since 2010 are absent from today's constituent list. Projecting today's winners backward in time means picking stocks with hindsight.

**The question:** How many percentage points per year does survivorship bias add to an equity backtest? Is it a rounding error, or is it large enough to invalidate your results?

### Tasks

1. From `sp500_prices`, identify tickers with data starting significantly *after* 2010 — these are late entrants added after they'd already succeeded
2. Build two equal-weight portfolios: (a) **all** current constituents including late entrants, and (b) **only** tickers with complete data from 2010
3. Compute the annualized return difference — that's your survivorship bias estimate
4. Plot both cumulative return paths. Does the gap widen over time?

In [ ]:
# TODO: Identify late entrants — tickers with first_valid_index() after 2011-01-01
# TODO: Build two equal-weight portfolios (biased vs. survivors-only)
# TODO: Compute annualized return for each and the annual bias = difference
# TODO: Plot cumulative returns for both portfolios on the same chart

---
### ▶ Solution

In [ ]:
late_entry_threshold = pd.Timestamp("2011-01-01")
first_valid = sp500_prices.apply(lambda col: col.first_valid_index())

late_entrants = first_valid[first_valid > late_entry_threshold].sort_values()
full_history_tickers = first_valid[first_valid <= late_entry_threshold]

print(f"Downloaded: {sp500_prices.shape[1]} tickers")
print(f"Late entrants (data starts after 2011): {len(late_entrants)}")
print(f"Full-history tickers: {len(full_history_tickers)}")
print(f"\nSample late entrants:")
print(late_entrants.head(10))

Many of those late entrants are names you recognize — companies like Meta (IPO'd in 2012) or firms added to the S&P 500 after years of strong performance. That's the selection mechanism at work: the index committee adds winners and removes losers. If you backtest using today's list, you're implicitly saying "I would have bought these in 2010" — but you couldn't have, because some didn't exist yet and others hadn't proven themselves.

Now let's build both portfolios and measure what that selection bias costs in hard numbers.

In [ ]:
biased_prices = sp500_prices.ffill().dropna(axis=1, how="all")
biased_returns = biased_prices.pct_change().mean(axis=1)
biased_cumret = (1 + biased_returns).cumprod()

survivor_tickers = full_history_tickers.index.tolist()
survivor_prices = sp500_prices[survivor_tickers].ffill()
survivor_returns = survivor_prices.pct_change().mean(axis=1)
survivor_cumret = (1 + survivor_returns).cumprod()

n_years = (sp500_prices.index[-1] - sp500_prices.index[0]).days / 365.25
biased_ann = (biased_cumret.iloc[-1]) ** (1 / n_years) - 1
survivor_ann = (survivor_cumret.iloc[-1]) ** (1 / n_years) - 1
annual_bias = biased_ann - survivor_ann
cumulative_bias = (biased_cumret.iloc[-1] - 1) - (survivor_cumret.iloc[-1] - 1)

print(f"Biased portfolio annualized return:   {biased_ann:.2%}")
print(f"Survivor portfolio annualized return:  {survivor_ann:.2%}")
print(f"Annual survivorship bias:              {annual_bias:.2%}")
print(f"Cumulative bias over {n_years:.1f} years:       {cumulative_bias:.2%}")

That annual bias number deserves attention. A couple of percentage points per year sounds small in isolation, but compounding is relentless. Over a 15-year backtest, that annual wedge accumulates into a cumulative overstatement that would change any allocation decision.

Think about it from a portfolio manager's perspective: if you present a backtest showing 14% annualized returns and 2-3 of those points come from survivorship bias, you haven't found alpha — you've found an artifact. The PM will figure this out roughly 6 months after allocating capital, when real-world returns run below the backtest. That's a career-defining conversation. Let's see the divergence visually.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(biased_cumret.index, (biased_cumret - 1) * 100,
        label="Survivorship-biased (all current constituents)",
        color="#C62828", linewidth=1.2)
ax.plot(survivor_cumret.index, (survivor_cumret - 1) * 100,
        label="Survivors-only (complete data from 2010)",
        color="#1565C0", linewidth=1.2)
ax.axhline(0, color="gray", linestyle="--", alpha=0.3)
ax.set_xlabel("Date")
ax.set_ylabel("Cumulative Return (%)")
ax.set_title("Survivorship Bias in S&P 500 Equal-Weight Portfolios (2010-2025)")
ax.legend()
plt.tight_layout()
plt.show()

The two lines start together and slowly diverge — the gap accumulates gradually, which is exactly what makes survivorship bias so dangerous. A backtest that looks reasonable year-by-year can be systematically inflated over the full period.

Here's what the lecture *didn't* tell you: the bias isn't constant over time. The gap widens more sharply during and after bull markets. That makes sense — when the market is rising, companies being added to the index are the biggest winners (they got added *because* they won). During crashes, few companies are added, so the bias grows slowly. This means survivorship bias is *procyclical*: it inflates your good years more than your bad years, which also inflates your Sharpe ratio. A risk manager checking for this would need to decompose the bias by regime, not just report an annual average.

The practical fix? Point-in-time databases like CRSP record what the index looked like on each historical date — including companies that later failed. Free data from yfinance with today's ticker list is a starting point, not a finish line.

## Exercise 2: Corporate Action Forensics — Splits, Dividends, and the Slow Drift

The lecture showed Apple's dramatic 7-for-1 split — an 85% apparent price drop that was entirely mechanical. That's the easy case. But here's the question most people never ask: **what about the damage you *can't* see?**

Johnson & Johnson has never split in the last decade. But it pays a quarterly dividend of roughly $1.19 per share. Each ex-dividend date, the adjusted price shifts down by a tiny amount — barely visible on a daily chart. Over 15 years, those tiny shifts compound into a 25-30% cumulative distortion. If you're training a model on unadjusted JNJ prices, you're understating total returns by a quarter, and your model has no idea.

### Tasks

1. Using `ca_data` (Close + Adj Close) and `all_splits`, reconstruct nominal prices and compute the adjustment factor: `Nominal / Adj Close`
2. Plot the adjustment factor for all 5 tickers — where do you see step jumps vs. smooth drift?
3. Build a split detector: flag dates where the factor changes by >10%. What's your hit rate?
4. For JNJ: compute cumulative dividend drift over the full period
5. Compute daily return errors (nominal vs. adjusted) for all 5 tickers

In [ ]:
# TODO: Reconstruct nominal prices by reversing cumulative split factors from all_splits
# TODO: Compute adj_factor = nominal / adj_close for each ticker and plot
# TODO: Build split detector (>10% daily change in factor), compute JNJ drift
# TODO: Measure return RMSE between nominal-based and adjusted-based returns

---
### ▶ Solution

In [ ]:
KNOWN_SPLITS = {"AAPL": ["2014-06-09", "2020-08-31"], "TSLA": ["2020-08-31", "2022-08-25"],
                "GE": ["2021-08-02"], "NVDA": ["2021-07-20", "2024-06-10"], "JNJ": []}
close = ca_data["Close"]
adj_close = ca_data["Adj Close"]
adj_factors = pd.DataFrame(index=close.index)
nominal_prices = pd.DataFrame(index=close.index)

for ticker in CA_TICKERS:
    close_t, adj_t = close[ticker].dropna(), adj_close[ticker].dropna()
    csf = pd.Series(1.0, index=close_t.index)
    if ticker in all_splits.columns:
        for sd, ratio in all_splits[ticker].dropna().items():
            if ratio > 0 and sd >= close_t.index[0]:
                csf.loc[:sd - pd.Timedelta(days=1)] *= ratio
    nominal_prices[ticker] = close_t * csf
    adj_factors[ticker] = (close_t * csf) / adj_t

adj_factors = adj_factors.ffill()

The core trick is reconstructing truly nominal prices. The yfinance `Close` with `auto_adjust=False` is confusingly already split-adjusted — it just hasn't been dividend-adjusted. To recover what someone actually saw on their screen in 2013, we reverse the cumulative split factor by walking backward through the split history. The resulting adjustment factor encodes the *entire* corporate action biography: every split as a step function, every dividend as a tiny decrement.

Let's plot all five and see how radically different their stories are.

In [ ]:
COLORS = {"AAPL": "#1565C0", "TSLA": "#E65100", "GE": "#2E7D32",
          "NVDA": "#C62828", "JNJ": "#6A1B9A"}
fig, axes = plt.subplots(len(CA_TICKERS), 1, figsize=(14, 10), sharex=True)
for ax, ticker in zip(axes, CA_TICKERS):
    factor = adj_factors[ticker].dropna()
    ax.plot(factor.index, factor.values, color=COLORS[ticker], linewidth=1.0)
    ax.set_ylabel(ticker, fontsize=11, fontweight="bold", rotation=0, labelpad=40)
    ax.grid(alpha=0.2)
    for sd in KNOWN_SPLITS.get(ticker, []):
        ts = pd.Timestamp(sd)
        if factor.index.min() <= ts <= factor.index.max():
            ax.axvline(ts, color="gray", linestyle="--", linewidth=0.7, alpha=0.6)
axes[0].set_title("Corporate Action Adjustment Factors (Nominal / Adj Close, 2010-2025)")
axes[-1].set_xlabel("Date")
plt.tight_layout()
plt.show()

Five tickers, five completely different corporate action biographies. AAPL shows two clean downward steps — the 2014 7:1 and 2020 4:1 splits — plus gentle dividend drift between them. TSLA is the cleanest self-check: exactly 15x total factor (5 times 3), zero dividend drift, because Tesla has never paid a dividend.

GE is the messy one. The upward jump in 2021 is the 1:8 *reverse* split — the company consolidated shares during restructuring. Additional jumps around 2023-2024 are the GE HealthCare and GE Vernova spinoffs, which yfinance records as split-like adjustments. Real corporate histories are not always clean textbook cases.

JNJ at the bottom: no steps, just a smooth downward drift from roughly 1.6 to 1.0. That's 15 years of quarterly dividends compounding into substantial distortion. Let's build the split detector and quantify JNJ's slow bleed.

In [ ]:
SPLIT_THRESHOLD = 0.10
detected_splits = {}
for ticker in CA_TICKERS:
    chg = adj_factors[ticker].pct_change().abs()
    detected_splits[ticker] = [d.strftime("%Y-%m-%d") for d in chg[chg > SPLIT_THRESHOLD].index]

for ticker in CA_TICKERS:
    known, detected = KNOWN_SPLITS[ticker], detected_splits[ticker]
    print(f"{ticker}:  Known: {known}")
    print(f"         Detected: {detected}")
    if known:
        hits = sum(1 for k in known if any(
            abs((pd.Timestamp(k) - pd.Timestamp(d)).days) <= 3 for d in detected))
        print(f"         Hit rate: {hits}/{len(known)}")

The detector catches every known split — 100% hit rate with a 10% threshold. But look at GE: we listed 1 known split, yet the detector flagged 3 events. Those "extra" detections aren't false positives — they're real corporate actions (the HealthCare and Vernova spinoffs) that we didn't list. A production split detector would need to distinguish splits from spinoffs by checking whether a new ticker appeared on the same date.

JNJ correctly triggers zero detections. Each quarterly dividend is too small to cross the 10% threshold — exactly what makes dividends dangerous. Too small to detect per-event, too large to ignore in aggregate.

In [ ]:
jnj_factor = adj_factors["JNJ"].dropna()
cumulative_drift = (jnj_factor.iloc[0] / jnj_factor.iloc[-1]) - 1
n_yrs_jnj = (jnj_factor.index[-1] - jnj_factor.index[0]).days / 365.25

print(f"JNJ Dividend Drift ({n_yrs_jnj:.1f} years):")
print(f"  Adj factor 2010: {jnj_factor.iloc[0]:.4f}  |  2024: {jnj_factor.iloc[-1]:.4f}")
print(f"  Cumulative drift: {cumulative_drift:.2%}")
print(f"  Annual drift:     {(1 + cumulative_drift)**(1/n_yrs_jnj) - 1:.2%}")

Over roughly 15 years, JNJ's adjustment factor drifted by 20-30% purely from dividends. If you trained a model on unadjusted prices, your model's version of JNJ's history would understate total returns by that entire amount. For a stock returning 8-10% annualized, a 2% annual dividend distortion is not a footnote — it's a quarter of the signal. Now let's measure the return-level damage across all five tickers.

In [ ]:
return_errors = {}
for ticker in CA_TICKERS:
    nom_ret = nominal_prices[ticker].dropna().pct_change()
    adj_ret = adj_close[ticker].dropna().pct_change()
    err = (nom_ret - adj_ret).dropna()
    return_errors[ticker] = {"rmse": np.sqrt((err**2).mean()),
        "max_abs": err.abs().max(), "n_catastrophic": (err.abs() > 0.5).sum()}

print("Return Error Analysis (Nominal vs. Adjusted):")
for t, m in return_errors.items():
    print(f"  {t}: RMSE={m['rmse']:.4f}  Max={m['max_abs']:.2%}  "
          f"Days >50% err: {m['n_catastrophic']}")

The return errors reveal the two-speed nature of corporate action damage. For stocks with splits, the max absolute error exceeds 50% — often 80-90% — on split dates. A model that sees an 85% single-day loss will weight it heavily, learning a pattern that never happened. JNJ's profile is the opposite: no catastrophic days, but persistent small RMSE from daily dividend adjustments — death by a thousand cuts.

Here's the insight the lecture didn't give you: **the difficulty of detecting a corporate action is inversely proportional to its frequency.** Splits are rare and dramatic — easy to flag, easy to fix. Dividends are frequent and tiny — nearly impossible to detect from price data alone, yet they compound into the *larger* cumulative distortion. JNJ's 20-30% drift dwarfs any single split error in aggregate impact. In production, you don't detect dividends — you use a vendor that handles them. The real skill is knowing *when* to use adjusted vs. unadjusted: level analysis (options pricing, dollar volume) needs unadjusted; return analysis needs adjusted. Using the wrong one is always silent and always expensive.

## Exercise 3: Data Quality Gauntlet — How Clean Is "Clean" Data?

Corporate actions are a known problem with known solutions. The scarier issues are the ones hiding in "clean" data from a reputable source. You've got 10 years of daily OHLCV for 20 well-known stocks from yfinance — Apple, Microsoft, JP Morgan, the S&P 500 ETF. **How many quality issues can you find, and what do they cost a simple strategy?**

Think of this as a pre-flight checklist. A pilot doesn't skip the inspection because the plane looked fine yesterday.

### Tasks

1. **Missing data audit:** Count expected trading days (NYSE calendar) vs. actual rows. Flag gaps > 5 days.
2. **Stale price detection:** Consecutive identical Close prices (4 decimals). Legitimate or error?
3. **Outlier returns:** Flag |return| > 15%. Classify: genuine (COVID), artifact (split), or review.
4. **OHLC consistency:** Verify `High >= max(Open, Close)` and `Low <= min(Open, Close)`. Count violations.
5. **Impact:** Buy-and-hold on raw vs. cleaned data (forward-filled, capped ±30%). Worst-case damage?

In [ ]:
# TODO: Completeness ratio per ticker (expected NYSE trading days vs. actual rows)
# TODO: Stale prices, outlier returns, OHLC consistency checks
# TODO: Impact assessment — raw vs. cleaned buy-and-hold returns

---
### ▶ Solution

In [ ]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
expected_dates = pd.date_range(start="2010-01-01", end="2025-01-01", freq=us_bd)
n_expected = len(expected_dates)
dq_close = dq_data["Close"]
completeness = {}

print(f"Expected trading days: {n_expected}\n")
for ticker in DQ_TICKERS:
    n_actual = dq_close[ticker].dropna().shape[0]
    ratio = n_actual / n_expected
    completeness[ticker] = {"actual": n_actual, "expected": n_expected, "ratio": ratio}
    if ratio < 0.99:
        print(f"  {ticker}: {n_actual}/{n_expected} = {ratio:.2%} (INCOMPLETE)")

Most large-cap stocks show near-perfect completeness. The tickers below 99% are revealing: META didn't go public until 2012, and any name that underwent major restructuring shows gaps. A 1-day gap might be a data glitch; a multi-week gap usually means a suspension, merger transition, or the stock didn't exist yet. Forward-filling across a 20-day gap fabricates data — that's why your pipeline needs configurable fill limits.

Now the subtler checks. Stale prices and outlier returns test different kinds of data integrity.

In [ ]:
stale_counts = {}
for ticker in DQ_TICKERS:
    rounded = dq_close[ticker].dropna().round(4)
    stale_counts[ticker] = (rounded == rounded.shift(1)).sum()
total_stale = sum(stale_counts.values())

print("Stale prices (consecutive identical Close):")
for t in sorted(stale_counts, key=stale_counts.get, reverse=True)[:5]:
    if stale_counts[t] > 0: print(f"  {t}: {stale_counts[t]} days")
print(f"Total: {total_stale}")

Stale prices are a subtle corruption — a model interpreting zero return as "no news" when the reality is "data missing" learns the wrong lesson. For the liquid names in our universe most are benign, but the pattern flags real issues in less liquid periods. Now outlier returns — March 2020 should light up.

In [ ]:
outliers = []
for ticker in DQ_TICKERS:
    rets = dq_close[ticker].pct_change()
    for date, ret in rets[rets.abs() > 0.15].dropna().items():
        outliers.append({"ticker": ticker, "date": date, "return": ret})
outliers_df = pd.DataFrame(outliers)

covid_s, covid_e = pd.Timestamp("2020-03-01"), pd.Timestamp("2020-04-01")
outliers_df["class"] = outliers_df.apply(
    lambda r: "genuine (COVID)" if covid_s <= r["date"] <= covid_e
    else ("artifact (likely split)" if abs(r["return"]) > 0.5 else "review"), axis=1)

print(f"Outlier returns (|r| > 15%): {len(outliers_df)}")
print(outliers_df["class"].value_counts())

The classifier reveals a clean taxonomy. COVID crash outliers are genuine and should stay in training data as real tail risk. Artifacts (>50% returns) are split-related fiction. The "review" bucket is the hard part: a 20% earnings-day move for NFLX might be real, while a 15% SPY jump would be unprecedented. Domain knowledge can't be fully automated. Now structural consistency and the final cost assessment.

In [ ]:
ohlc_violations = {}
for ticker in DQ_TICKERS:
    o, h, l, c = [dq_data[col][ticker].dropna() for col in ["Open","High","Low","Close"]]
    v = dq_data["Volume"][ticker].dropna()
    idx = o.index.intersection(h.index).intersection(l.index).intersection(c.index)
    o, h, l, c, v = o[idx], h[idx], l[idx], c[idx], v[idx]
    ohlc_violations[ticker] = ((h<o)|(h<c)).sum() + ((l>o)|(l>c)).sum() + (v<=0).sum()

print("OHLC violations:")
for t in sorted(ohlc_violations, key=ohlc_violations.get, reverse=True)[:5]:
    if ohlc_violations[t] > 0: print(f"  {t}: {ohlc_violations[t]}")
print(f"Total: {sum(ohlc_violations.values())}")

OHLC violations are the canary in the coal mine — if `High < Close`, the data is structurally broken. Zero violations for large-cap stocks is the expected baseline; anything above zero warrants investigation. Let's measure the bottom-line return impact of all these quality issues combined.

In [ ]:
impact_results = []
for ticker in DQ_TICKERS:
    raw_p = dq_close[ticker].dropna()
    clean_p = dq_close[ticker].ffill(limit=5).dropna()
    clean_ret = clean_p.pct_change().dropna().clip(-0.30, 0.30)
    clean_recon = clean_p.iloc[0] * (1 + clean_ret).cumprod()
    ny = (raw_p.index[-1] - raw_p.index[0]).days / 365.25
    if ny > 0 and len(raw_p) > 1:
        r_ann = (raw_p.iloc[-1]/raw_p.iloc[0])**(1/ny) - 1
        c_ann = (clean_recon.iloc[-1]/clean_p.iloc[0])**(1/ny) - 1
        impact_results.append({"ticker": ticker, "raw_ann": r_ann,
                               "cleaned_ann": c_ann, "impact": abs(r_ann - c_ann)})

impact_df = pd.DataFrame(impact_results).sort_values("impact", ascending=False)
print(impact_df.head(10).to_string(index=False))

The impact table is the punchline. For most large-cap stocks, the difference between raw and cleaned returns is small — often under 0.5% annualized. But the worst-case ticker shows a meaningfully larger discrepancy, with return capping absorbing split artifacts that would otherwise distort total returns.

Here's what this exercise reveals that the lecture couldn't: **data quality is not binary — it's a spectrum, and it varies by stock.** The same data source gives near-perfect data for AAPL and materially worse data for names with messy corporate histories. "yfinance data is fine" and "yfinance data is unreliable" are both wrong. The correct statement: data quality depends on the ticker, and you need automated per-ticker checks to know which ones you can trust.

This is exactly what the homework asks you to build. At a real fund, per-ticker quality grades determine whether a stock enters the research universe or gets excluded. A junior quant data engineer's first task is often building this kind of health-check pipeline — and the PM's first question about any dataset is always "what's the quality score?"

## What We Discovered

- **Survivorship bias is a measurable tax on every backtest using a current-day stock list.** The annual bias of 1-4% compounds into double-digit cumulative overstatement, and it's *procyclical* — inflating bull-market returns more than bear-market returns, which also inflates the Sharpe ratio.

- **Corporate action damage operates at two speeds.** Splits are loud, rare, and 100% detectable with a simple threshold. Dividends are silent, frequent, and undetectable from price data alone — yet JNJ's 20-30% cumulative drift shows they can be the *larger* distortion.

- **Detection difficulty is inversely proportional to frequency.** The 85% return error from an AAPL split is impossible to miss. The 2% annual drag from dividend non-adjustment requires computing the adjustment factor explicitly. Automation catches what eyeballs miss.

- **Data quality is a per-ticker property, not a per-source property.** The same yfinance download gives near-perfect data for SPY and materially worse data for complex corporate histories. Any pipeline without per-ticker quality scores is flying blind.

In the homework, you'll build the `FinancialDataLoader` class that scales these checks to 50+ tickers — downloading, cleaning, validating, and scoring automatically. The patterns you've seen in 5 and 20 stocks get more interesting at 200.